<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtT5-XL-BFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Important Notes:
1. ProtT5-XL-BFD has both encoder and decoder, for feature extraction we only load and use the encoder part.
2. Loading only the encoder part, reduces the inference speed and the GPU memory requirements by half.
3. In order to use ProtT5-XL-BFD encoder, you must install the latest huggingface transformers version from their GitHub repo, or at least the bellow mentioned commit.
4. If you are intersted in both the encoder and decoder, you should use TFT5Model rather than TFT5EncoderModel.

<h3>Extracting protein sequences' features using ProtT5-XL-BFD pretrained-model</h3>


**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q SentencePiece git+https://github.com/huggingface/transformers.git@40ecaf0c2b1c0b3894e9abf619f32472c5a3b3ca

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 890kB 49.4MB/s 
     |████████████████████████████████| 2.9MB 54.1MB/s 


In [2]:
from transformers import TFT5EncoderModel, T5Tokenizer
import numpy as np
import re
import gc

<b>2. Load the vocabulary and ProtT5-XL-BFD Model<b>

In [3]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_bfd", do_lower_case=False )

In [4]:
model = TFT5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_bfd", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5EncoderModel: ['decoder.block.1.layer.1.layer_norm.weight', 'decoder.block.18.layer.2.DenseReluDense.wo.weight', 'decoder.block.17.layer.0.SelfAttention.v.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.10.layer.2.DenseReluDense.wi.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.8.layer.0.SelfAttention.v.weight', 'decoder.block.22.layer.2.layer_norm.weight', 'decoder.block.10.layer.1.EncDecAttention.k.weight', 'decoder.block.20.layer.1.layer_norm.weight', 'decoder.block.9.layer.2.layer_norm.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.9.layer.2.DenseReluDense.wo.weight', 'decoder.block.8.layer.0.SelfAttention.o.weight', 'decoder.block.6.layer.1.EncDecAttention.o.weight', 'decoder.block.11.layer.1.EncDecAttention.v.weight', 'decoder.block.19.layer.0.SelfAttention.v.weight', 'decoder.block.1.layer.0.SelfAttention.o.weig

In [5]:
gc.collect()

0

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [8]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")

In [9]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [10]:
embedding = model(input_ids)

In [11]:
embedding = np.asarray(embedding.last_hidden_state)

In [12]:
attention_mask = np.asarray(attention_mask)

<b>7. Remove padding (\<pad\>) and special tokens (\</s\>) that is added by ProtT5-XL-BFD model<b>

In [13]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

In [14]:
print(features)

[array([[ 0.29069257, -0.20804922, -0.21934749, ...,  0.33846596,
         0.46031442, -0.16942428],
       [ 0.20362839, -0.12995914, -0.17139854, ...,  0.10642263,
        -0.37738237,  0.05401217],
       [ 0.03736717, -0.08506598, -0.23616521, ...,  0.25152585,
         0.1333862 ,  0.03063789],
       ...,
       [ 0.47787994, -0.23029439, -0.10682338, ...,  0.40571734,
         0.52513736,  0.19175257],
       [ 0.07990429,  0.07094713, -0.08554702, ...,  0.2494752 ,
         0.36538345, -0.45506254],
       [ 0.09110186,  0.17046821,  0.42079183, ...,  0.25626653,
         0.02010926, -0.11016779]], dtype=float32), array([[ 0.32152936, -0.09309337, -0.15645395, ..., -0.02557738,
         0.06829377, -0.15527725],
       [ 0.10843595, -0.06435563,  0.15844898, ...,  0.05685001,
        -0.16503637,  0.15184814],
       [ 0.21142836,  0.11896314,  0.08190013, ...,  0.2332427 ,
         0.37030575, -0.23370315],
       [ 0.59764427, -0.07505642, -0.15460739, ...,  0.4994429 ,
     